In [1]:
import warnings

import pandas as pd
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

In [2]:
warnings.filterwarnings('ignore')
pd.set_option('max_columns', None)
spark = SparkSession.builder.appName("modelovaniNehod").getOrCreate()

21/12/22 17:43:07 WARN Utils: Your hostname, macusers-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.131 instead (on interface en0)
21/12/22 17:43:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/22 17:43:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.parquet("data/4_processed")
df.limit(5).toPandas()

,features,label
0,"[104.52085919440314, 3.962358629522141, 0.0, 4...",1.0
1,"[104.52085919440314, 4.129996879232693, 0.0, 4...",0.0
2,"[104.52085919440314, 4.210006043867275, 0.0, 4...",0.0
3,"[104.52085919440314, 4.17952636210172, 0.0, 4....",1.0
4,"[105.53097940024936, 3.825200061577144, 0.0, 4...",0.0


In [45]:
with open('data/feature_list.txt', 'r') as f:
    feature_cols = f.read().splitlines()

# Split data

In [4]:
trainData, testData = df.randomSplit([0.7, 0.3])
print("trainData count is {}, testDataCount is: {}".format(trainData.count(), testData.count()))

trainData count is 4153059, testDataCount is: 1778366


# Prepare estimators

In [5]:
stages = []

# DECISION TREE CLASSIFIER
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", predictionCol="DTprediction", rawPredictionCol="DTRawPrediction", probabilityCol="DTProbability")
stages.append(dt)

# RANDOM FOREST CLASSIFIER
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="label", featuresCol="features", predictionCol="RFprediction", rawPredictionCol="RFRawPrediction", probabilityCol="RFProbability", numTrees=10)
stages.append(rf)

# GBT CLASSIFIER
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(labelCol="label", featuresCol="features", predictionCol="GBTprediction", maxIter=10)
stages.append(gbt)

# LOGISTIC REGRESSION
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol="features", labelCol="label", predictionCol="LRprediction", probabilityCol="LRProbability", rawPredictionCol="LRRawPrediction", family="binomial", threshold=0.5)
stages.append(lr)

# NAIVE BAYES
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(smoothing=1.0, modelType="multinomial", featuresCol="features", labelCol="label", predictionCol="NBprediction", probabilityCol="NBProbability", rawPredictionCol="NBRawPprediction")
stages.append(nb)

# Predict

In [6]:
pipeline = Pipeline(stages = stages)
predictionModel = pipeline.fit(trainData)
predictions = predictionModel.transform(testData)
predictions.createOrReplaceTempView("predictions")

21/12/22 17:44:16 WARN MemoryStore: Not enough space to cache rdd_48_6 in memory! (computed 12.8 MiB so far)
21/12/22 17:44:16 WARN BlockManager: Persisting block rdd_48_6 to disk instead.
21/12/22 17:44:16 WARN MemoryStore: Not enough space to cache rdd_48_7 in memory! (computed 19.7 MiB so far)
21/12/22 17:44:16 WARN BlockManager: Persisting block rdd_48_7 to disk instead.
21/12/22 17:44:21 WARN MemoryStore: Not enough space to cache rdd_48_4 in memory! (computed 29.5 MiB so far)
21/12/22 17:44:21 WARN BlockManager: Persisting block rdd_48_4 to disk instead.
21/12/22 17:44:21 WARN MemoryStore: Not enough space to cache rdd_48_0 in memory! (computed 44.3 MiB so far)
21/12/22 17:44:21 WARN BlockManager: Persisting block rdd_48_0 to disk instead.
21/12/22 17:44:21 WARN MemoryStore: Not enough space to cache rdd_48_2 in memory! (computed 44.3 MiB so far)
21/12/22 17:44:21 WARN BlockManager: Persisting block rdd_48_2 to disk instead.
21/12/22 17:44:22 WARN MemoryStore: Not enough space to

# Evaluate

In [7]:
predictions.limit(5).toPandas()

,features,label,DTRawPrediction,DTProbability,DTprediction,RFRawPrediction,RFProbability,RFprediction,rawPrediction,probability,GBTprediction,LRRawPrediction,LRProbability,LRprediction,NBRawPprediction,NBProbability,NBprediction
0,"[102.92593255359333, 3.905209226211726, 0.0, 4...",1.0,"[30779.0, 116528.0]","[0.2089445851181546, 0.7910554148818454]",1.0,"[2.4021040293614733, 7.597895970638527]","[0.24021040293614732, 0.7597895970638527]",1.0,"[-0.901794206178098, 0.901794206178098]","[0.14141481096729128, 0.8585851890327088]",1.0,"[-2.1828479567991, 2.1828479567991]","[0.1013013570807014, 0.8986986429192986]",1.0,"[-287.48733605003764, -287.22095549893794]","[0.43379587932429137, 0.5662041206757085]",1.0
1,"[102.92593255359333, 4.000458231729085, 0.0, 4...",0.0,"[16629.0, 24828.0]","[0.4011144076995441, 0.5988855923004559]",1.0,"[3.4771701032012, 6.522829896798799]","[0.34771701032012003, 0.65228298967988]",1.0,"[-0.38586842532636545, 0.38586842532636545]","[0.31610351040381773, 0.6838964895961823]",1.0,"[-0.7411093923378038, 0.7411093923378038]","[0.32276159785290637, 0.6772384021470936]",1.0,"[-286.3366729817417, -286.1653199815605]","[0.4572662606278224, 0.5427337393721775]",1.0
2,"[102.92593255359333, 4.042367794156723, 0.0, 4...",1.0,"[16629.0, 24828.0]","[0.4011144076995441, 0.5988855923004559]",1.0,"[3.4771701032012, 6.522829896798799]","[0.34771701032012003, 0.65228298967988]",1.0,"[-0.3154662340043357, 0.3154662340043357]","[0.3472991335925465, 0.6527008664074535]",1.0,"[-0.22984896085900797, 0.22984896085900797]","[0.44278941050571774, 0.5572105894942823]",1.0,"[-290.5571628105681, -290.4365119600709]","[0.46987382314110365, 0.5301261768588963]",1.0
3,"[102.92593255359333, 4.046177754377418, 0.0, 4...",0.0,"[175836.0, 30294.0]","[0.8530344927958085, 0.14696550720419152]",0.0,"[7.546617126285232, 2.453382873714768]","[0.7546617126285232, 0.2453382873714768]",0.0,"[1.0589245858923566, -1.0589245858923566]","[0.8926259577066441, 0.1073740422933559]",0.0,"[2.4035824577781524, -2.4035824577781524]","[0.9171000762065654, 0.08289992379343458]",0.0,"[-300.0677363099973, -300.1001850649354]","[0.5081114770178254, 0.4918885229821745]",0.0
4,"[102.92593255359333, 4.0614175952601945, 0.0, ...",0.0,"[48740.0, 20650.0]","[0.7024066868424845, 0.29759331315751547]",0.0,"[4.907409028878005, 5.092590971121995]","[0.49074090288780053, 0.5092590971121995]",1.0,"[0.3616243460196828, -0.3616243460196828]","[0.6733219996501077, 0.32667800034989225]",0.0,"[0.6837328385535946, -0.6837328385535946]","[0.664571318420261, 0.33542868157973904]",0.0,"[-294.87571721368244, -294.8125878722156]","[0.4842229040084301, 0.5157770959915698]",1.0


In [8]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# DECISION TREE
dtEvaluator = BinaryClassificationEvaluator(metricName="areaUnderROC", labelCol="label", rawPredictionCol="DTRawPrediction")
print("DECISION TREE Area under ROC: " + str(dtEvaluator.evaluate(predictions)))

#RANDOM FOREST
rfEvaluator = BinaryClassificationEvaluator(metricName="areaUnderROC", labelCol="label", rawPredictionCol="RFRawPrediction")
print("RANDOM FOREST Area under ROC: " + str(rfEvaluator.evaluate(predictions)))

#GBT
gbtEvaluator = BinaryClassificationEvaluator(metricName="areaUnderROC", labelCol="label", rawPredictionCol="rawPrediction")
print("GBT Area under ROC: " + str(gbtEvaluator.evaluate(predictions)))

#LOGISTIC REGRESSION
blrEvaluator = BinaryClassificationEvaluator(metricName="areaUnderROC", labelCol="label", rawPredictionCol="LRRawPrediction")
print("LOGISTIC REGRESSION Area under ROC: " + str(blrEvaluator.evaluate(predictions)))

#NAIVE BAYES
nbEvaluator = BinaryClassificationEvaluator(metricName="areaUnderROC", labelCol="label", rawPredictionCol="NBprediction")
print("NAIVE BAYES Area under ROC: " + str(nbEvaluator.evaluate(predictions)))

DECISION TREE Area under ROC: 0.7065426767405321


RANDOM FOREST Area under ROC: 0.8192341375482894


GBT Area under ROC: 0.8518099314621091


LOGISTIC REGRESSION Area under ROC: 0.8652888580655989


NAIVE BAYES Area under ROC: 0.6720437188811474


In [26]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics, MulticlassMetrics

def get_metrics(prefix):
    predictionAndTarget = predictions.select(['label', prefix + 'prediction'])

    # Create both evaluators
    metrics_binary = BinaryClassificationMetrics(predictionAndTarget.rdd.map(tuple))
    metrics_multi = MulticlassMetrics(predictionAndTarget.rdd.map(tuple))

    acc = metrics_multi.accuracy
    f1 = metrics_multi.fMeasure(1.0)
    precision = metrics_multi.precision(1.0)
    recall = metrics_multi.recall(1.0)
    auc = metrics_binary.areaUnderROC
    
    return dict(
        acc = acc,
        f1 = f1,
        precision = precision,
        recall = recall,
        auc = auc
    )

In [ ]:
estimators = ['DT', 'RF', 'GBT', 'LR', 'NB']
performances = dict()

for estimator in estimators:
    print(estimator)
    performances[estimator] = get_metrics(estimator)

In [32]:
pd.DataFrame.from_dict(performances).T.sort_values('f1', ascending=False)

,acc,f1,precision,recall,auc
LR,0.778986,0.789953,0.794212,0.785739,0.778572
GBT,0.767129,0.777376,0.776975,0.777778,0.766622
NB,0.683145,0.750448,0.910443,0.638281,0.726739
RF,0.732077,0.749779,0.767102,0.733221,0.731957
DT,0.736540,0.740995,0.720204,0.763022,0.736858


## Logistic regression coefficients

In [33]:
lrModel = lr.fit(trainData)

21/12/22 18:09:13 WARN MemoryStore: Not enough space to cache rdd_876_6 in memory! (computed 17.0 MiB so far)
21/12/22 18:09:13 WARN BlockManager: Persisting block rdd_876_6 to disk instead.
21/12/22 18:09:17 WARN MemoryStore: Not enough space to cache rdd_876_3 in memory! (computed 33.0 MiB so far)
21/12/22 18:09:17 WARN BlockManager: Persisting block rdd_876_3 to disk instead.
21/12/22 18:09:17 WARN MemoryStore: Not enough space to cache rdd_876_0 in memory! (computed 33.0 MiB so far)
21/12/22 18:09:17 WARN BlockManager: Persisting block rdd_876_0 to disk instead.
21/12/22 18:09:17 WARN MemoryStore: Not enough space to cache rdd_876_1 in memory! (computed 33.0 MiB so far)
21/12/22 18:09:17 WARN BlockManager: Persisting block rdd_876_1 to disk instead.
21/12/22 18:09:17 WARN MemoryStore: Not enough space to cache rdd_876_2 in memory! (computed 33.0 MiB so far)
21/12/22 18:09:17 WARN BlockManager: Persisting block rdd_876_2 to disk instead.
21/12/22 18:09:17 WARN MemoryStore: Not enoug

In [58]:
pd.DataFrame.from_dict(dict(zip(feature_cols, lrModel.coefficients)), orient='index').sort_values(0, ascending=False)

,0
Elo hráče před,2.774835
Max elo,0.449690
Rok narození,0.054084
Max elo_soupeř,0.039871
Elo nejlepšího poraženého soupeře,0.032558
Hráč je žena,0.003119
Den v týdnu,0.002118
Soupeř je žena,0.000159
Víkend,0.000000
obdobi,0.000000


<hr>

# Scikit model tuning

In [ ]:
lr = LogisticRegression(n_jobs=-1, random_state=42)
param_dist = {
    "penalty": ['l2', 'none'],
    "max_iter": [5, 10, 50, 100, 1000, 10000], 
    "C": [0.5, 1.0, 1.5],
    "fit_intercept":[True, False],
    "solver":['newton-cg', 'lbfgs', 'sag']

}
gs = GridSearchCV(lr, param_dist, error_score='raise')
gs.fit(x_train, y_train.values.ravel())

In [ ]:
gs.best_params_

In [ ]:
gs.best_score_

Diky hyperparameter tuningu jsme dosahli zlepseni o 0,8 pp.

In [ ]:
acc(y_test, gs.predict(x_test))

<hr><hr>

# Neuronove site (Keras)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(512, activation="relu", input_shape=(x_train.shape[1],)),
    keras.layers.Dropout(0.4),

    keras.layers.Dense(256, activation="relu", input_shape=(3,)),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(1, activation="sigmoid"),
])

model.compile(loss="binary_crossentropy",
             optimizer="adam",
             metrics=["accuracy"])

history = model.fit(x_train, y_train, epochs=20, validation_data=[x_test, y_test], callbacks=[keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])

In [ ]:
pd.DataFrame(history.history)[['loss', 'val_loss']].iloc[0:].plot()

Nase neuronove site dosahly nejvyssi accuracy 76,6%, tedy o 0,5pp horsi vysledek nez nejlepsi logisticka regrese. Zkusime model neuronovych siti poladit pomoci GridSearch.

# Tuning neuronovych siti

In [ ]:
def create_model(optimizer="adam", dropout=0.1, init='uniform', nbr_features=9, dense_nparams=256):
    model = Sequential()
    model.add(Dense(dense_nparams, activation='relu', input_shape=(nbr_features,), kernel_initializer=init,)) 
    model.add(Dropout(dropout), )
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=["accuracy"])
    return model

keras_estimator = KerasClassifier(build_fn=create_model, verbose=1)

param_grid = {
    'epochs': [5],
    'dense_nparams': [32, 256, 512],
    'init': [ 'uniform', 'zeros', 'normal', ], 
    'batch_size':[2, 16, 32],
    # 'kc__optimizer':['RMSprop', 'Adam', 'Adamax', 'sgd'],
    'dropout': [0.4, 0.2, 0]
}

grid = GridSearchCV(estimator=keras_estimator,  
                    n_jobs=-1, 
                    verbose=1,
                    return_train_score=True,
                    cv=5,
                    param_grid=param_grid,)

grid.fit(x_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

In [ ]:
acc(y_test, grid.best_estimator_.predict(x_test))

In [ ]:
nn1 = grid.best_estimator_

<hr>

In [ ]:
def create_model(optimizer="adam", dropout=0.1, init='uniform', nbr_features=9, dense_nparams=256):
    model = Sequential()
    model.add(Dense(dense_nparams, activation='relu', input_shape=(nbr_features,), kernel_initializer=init,)) 
    model.add(Dropout(dropout), )

    model.add(Dense(dense_nparams, activation='relu')) 
    model.add(Dropout(dropout), )
    
    model.add(Dense(dense_nparams, activation='relu')) 
    model.add(Dropout(dropout), )

    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=["accuracy"])
    return model

keras_estimator = KerasClassifier(build_fn=create_model, verbose=1)

param_grid = {
    'epochs': [5],
    'dense_nparams': [32, 256, 512],
    'init': ['normal'], 
    'batch_size':[32],
    'dropout': [0.4, 0.2, 0]
}

grid = GridSearchCV(estimator=keras_estimator,  
                    n_jobs=-1, 
                    verbose=1,
                    return_train_score=True,
                    cv=5,
                    param_grid=param_grid,)

grid.fit(x_train, y_train)



In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

In [ ]:
acc(y_test, grid.best_estimator_.predict(x_test))

In [ ]:
nn2 = grid.best_estimator_

<hr>

In [ ]:
keras_estimator = KerasClassifier(build_fn=create_model, verbose=1)

param_grid = {
    'epochs': [5],
    'dense_nparams': [16, 32, 64],
    'init': ['normal'], 
    'batch_size':[32, 64],
    'dropout': [0.2]
}
grid = GridSearchCV(estimator=keras_estimator,  
                    n_jobs=-1, 
                    verbose=1,
                    return_train_score=True,
                    cv=5,
                    param_grid=param_grid,)
grid.fit(x_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
acc(y_test, grid.best_estimator_.predict(x_test))

In [ ]:
nn3 = grid.best_estimator_

<hr>

# Shrnuti

Model logisticke regrese po tuningu dosahl presnosti 75,48 % na testovacim setu. Nejlepsi model neuronovych siti dosahl 76,52 %, tedy o 1,04pp lepsi vysledek. Otazka je, zda tento rozdil stoji za vypocetni vykon a cas potrebny k natrenovani site. Dle meho nazoru nestoji.

Popis nejlepsiho modelu:

{'batch_size': 32,
 'dense_nparams': 32,
 'dropout': 0.2,
 'epochs': 5,
 'init': 'normal'}

Sekvencni model Keras se tremi huste propojenymi vrstvami o 32 parametrech, velikosti davky 32 a naslednym 20% dropoutem.